## Notebook match and combine dataframes with the enrollment data

### going to use fuzzy matching to acomplish the task

In [56]:
#### New attempt now that the data is cleaned
import pandas as pd
import numpy as np
import difflib

### Read the enrollment table from MHSAA website - 2022 enrollment
mhsaa_df = pd.read_excel('data\collected.xlsx', sheet_name='2022_enrollment')
# mhsaa_df.columns
# rename the columns
mhsaa_df.rename(columns={ 'enrollment_total':'students',
       'enrollment_classification':'enrollment'}, inplace=True)
#mhsaa_df.head()
# ## dataframe looks good


# load the table with the high school field size data

# read the file
df = pd.read_csv('TEMP/tost.csv')

# select just the high school fields
hs_df = df[df['level'] == 'high_schools']



In [57]:
from fuzzywuzzy import fuzz
from fuzzywuzzy import process

hs_names = hs_df['field'].tolist()
enroll_names = mhsaa_df['school_name'].tolist()
id_list = mhsaa_df['school_id'].tolist()

# set the treshold for the fuzzy match
treshold = 80
mat1 = []


# output a list of the school_id from the mhsaa_df based on the fuzzy match
for i in hs_names:
    mat1.append(process.extract(i, enroll_names, limit=1))

hs_df['match'] = mat1
# get text of first value in the tuple
hs_df['match'] = hs_df['match'].apply(lambda x: x[0][0])

# create new dataframe by merging the two dataframes
new_df = pd.merge(hs_df, mhsaa_df, left_on='match', right_on='school_name', how='left')


drop the columns that are not needed
### DROPPING THE POLYGON COLUMNS
new_df.drop(['foul','fop','level_x', 'match'], axis=1, inplace=True)

# # new_df.info()

## export csv to look at table
# new_df.to_csv('TEMP/new_df.csv')


C:\Users\Justin\AppData\Local\Temp\ipykernel_5064\3432091826.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  hs_df['match'] = mat1
C:\Users\Justin\AppData\Local\Temp\ipykernel_5064\3432091826.py:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  hs_df['match'] = hs_df['match'].apply(lambda x: x[0][0])


In [65]:
#### clean the new dataframe
#####

# drop the columns that are not needed
#### DROPPING THE POLYGON COLUMNS
# new_df.drop(['foul','fop','level_x', 'match'], axis=1, inplace=True)

# Rename some of the columns
new_df.rename(columns={'field':'map_name', 'school_id_y':'mhsaa_id', 'level_y':'level'}, inplace=True)

# Reorder the columns
new_df = new_df[['school_id', 
                'school_name', 
                'map_name', 
                'level', 
                'division', 
                'enrollment', 
                'students', 
                'foul_ft', 
                'fop_ft', 
                'foul_pct']]
new_df.info()
# rename the columns
# field:map_name
# school_id_y:mhsaa_id
# level_y:level

# ## Output the new dataframe to a csv file
# new_df.to_csv('TEMP/hs_divisions.csv')
# #### WORKING OUTPUT 



<class 'pandas.core.frame.DataFrame'>
Int64Index: 155 entries, 0 to 154
Data columns (total 10 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   school_id    155 non-null    int64  
 1   school_name  155 non-null    object 
 2   map_name     155 non-null    object 
 3   level        155 non-null    object 
 4   division     155 non-null    object 
 5   enrollment   155 non-null    int64  
 6   students     155 non-null    int64  
 7   foul_ft      155 non-null    float64
 8   fop_ft       155 non-null    float64
 9   foul_pct     155 non-null    float64
dtypes: float64(3), int64(3), object(4)
memory usage: 13.3+ KB


C:\Users\Justin\AppData\Local\Temp\ipykernel_5064\1798732687.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_df.rename(columns={'field':'map_name', 'school_id_y':'mhsaa_id', 'level_y':'level'}, inplace=True)


WORKING UP TO HERE - GOOD CLEAN OUTPUT OF ALL THE HS FIELDS WITH MHSAA DATA SCRAPED AND ADDED

In [81]:
###### merge the high school data frame back into the larger dataframe

df = pd.read_csv('TEMP/tost.csv')
df.info()
df.head()

## Drop all the high school rows
df = df[df['level'] != 'high_schools']
df.info()

## add all the rows from the new_df to the df
df = df.append(new_df, ignore_index=True)

# copy field to map_name if map_name is null
for i in range(len(df)):
    if pd.isnull(df['map_name'][i]):
        df['map_name'][i] = df['field'][i]

### 
df.info()

### Output the new dataframe to a csv file
df.to_csv('TEMP/merge_test.csv')

#### PROBLEM - THE home_plate values are lost along the way. 
# Need to fix that.

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 215 entries, 0 to 214
Data columns (total 8 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   field       215 non-null    object 
 1   foul        215 non-null    object 
 2   fop         215 non-null    object 
 3   level       215 non-null    object 
 4   home_plate  215 non-null    object 
 5   foul_ft     215 non-null    float64
 6   fop_ft      215 non-null    float64
 7   foul_pct    215 non-null    float64
dtypes: float64(3), object(5)
memory usage: 13.6+ KB
<class 'pandas.core.frame.DataFrame'>
Int64Index: 60 entries, 0 to 214
Data columns (total 8 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   field       60 non-null     object 
 1   foul        60 non-null     object 
 2   fop         60 non-null     object 
 3   level       60 non-null     object 
 4   home_plate  60 non-null     object 
 5   foul_ft     60 non-null     float64
 6

C:\Users\Justin\AppData\Local\Temp\ipykernel_5064\2569710641.py:12: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(new_df, ignore_index=True)
C:\Users\Justin\AppData\Local\Temp\ipykernel_5064\2569710641.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['map_name'][i] = df['field'][i]


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 215 entries, 0 to 214
Data columns (total 14 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   field        60 non-null     object 
 1   foul         60 non-null     object 
 2   fop          60 non-null     object 
 3   level        215 non-null    object 
 4   home_plate   60 non-null     object 
 5   foul_ft      215 non-null    float64
 6   fop_ft       215 non-null    float64
 7   foul_pct     215 non-null    float64
 8   school_id    155 non-null    float64
 9   school_name  155 non-null    object 
 10  map_name     215 non-null    object 
 11  division     155 non-null    object 
 12  enrollment   155 non-null    float64
 13  students     155 non-null    float64
dtypes: float64(6), object(8)
memory usage: 23.6+ KB


#### STOPPED HERE WEDNEDSAY NIGHT - 
# LOOKING AT OUTPUT IN TAB

In [ ]:
### Lookup school_id and school_name from enroll_df and copy it into the area_sqft_df

import difflib
area_sqft_df['school_name'] = area_sqft_df['field'].apply(lambda x: difflib.get_close_matches(x, enroll_df['school_name'], n=1, cutoff=0.6))
area_sqft_df['school_name'] = area_sqft_df['school_name'].apply(lambda x: x[0] if len(x) > 0 else np.nan)
new_df = enroll_df.merge(area_sqft_df, on='school_name', how='left')

## Drop row if there is no home plate location
new_df = new_df.dropna(subset=['home_plate'])

area_sqft_df.head()
area_sqft_df.info()
enroll_df.head()
new_df.info()
new_df.head()

## export to check
new_df.to_csv('test_merge_new_df.csv')


## Working above

### currently matching 159 of 222 records (n=1, cutoff=0.6)
#### was matching about 60 records with (cutoff=0.8)

In [ ]:
### cell to 

In [ ]:
### Lookup enrollment numbers and school id from the enrollment sheet and add them to the high school sheet

import difflib

high_school_df['match'] = high_school_df['School'].apply(lambda x: difflib.get_close_matches(x, enroll_df['school_name'].tolist(), n=1, cutoff=0.6))
high_school_df['match'] = high_school_df['match'].apply(lambda x: x[0] if len(x) > 0 else np.nan)
high_school_df['enrollment'] = high_school_df['match'].apply(lambda x: enroll_df[enroll_df['school_name'] == x]['enrollment_classification'].values[0] if type(x) == str else np.nan)
high_school_df['school_id'] = high_school_df['match'].apply(lambda x: enroll_df[enroll_df['school_name'] == x]['school_id'].values[0] if type(x) == str else np.nan)
high_school_df['Div'] = high_school_df['match'].apply(lambda x: enroll_df[enroll_df['school_name'] == x]['division'].values[0] if type(x) == str else np.nan)
high_school_df['enrollment'] = high_school_df['enrollment'].fillna(0)
high_school_df['school_id'] = high_school_df['school_id'].fillna(0)
high_school_df = high_school_df.drop(columns=['match'])
high_school_df.head()

high_school_df.describe()

# Export csv to check output

high_school_df.to_csv('data\high_school_df_test.csv')



In [ ]:
## Lookup enrollment numbers and school id from the enrollment sheet and add them to the area_sqft sheet

import difflib

area_sqft_df['match'] = area_sqft_df['field'].apply(lambda x: difflib.get_close_matches(x, enroll_df['school_name'].tolist(), n=1, cutoff=0.8))
area_sqft_df['match'] = area_sqft_df['match'].apply(lambda x: x[0] if len(x) > 0 else np.nan)
area_sqft_df['enrollment'] = area_sqft_df['match'].apply(lambda x: enroll_df[enroll_df['school_name'] == x]['enrollment_classification'].values[0] if type(x) == str else np.nan)
area_sqft_df['school_id'] = area_sqft_df['match'].apply(lambda x: enroll_df[enroll_df['school_name'] == x]['school_id'].values[0] if type(x) == str else np.nan)
area_sqft_df['Div'] = area_sqft_df['match'].apply(lambda x: enroll_df[enroll_df['school_name'] == x]['division'].values[0] if type(x) == str else np.nan)
area_sqft_df['enrollment'] = area_sqft_df['enrollment'].fillna(0)
area_sqft_df['school_id'] = area_sqft_df['school_id'].fillna(0)
area_sqft_df = area_sqft_df.drop(columns=['match'])
area_sqft_df.info()

## Export csv to check output
area_sqft_df.to_csv('area_sqft_df_test.csv')



## Code above works

### Improved lookups by cleaning up the names in the kml document

#### Haven't checked the results line by line yet

In [ ]:
##### THIS BLOCK IS KIND OF A MESS AND PROBABLY ISN'T GOING TO BE USED

### merge the dataframes
import difflib
# from fuzzywuzzy import fuzz
# from fuzzywuzzy import process


high_school_df['match'] = high_school_df['School'].str.lower()
enroll_df['match'] = enroll_df['school_name'].str.lower()
area_sqft_df['match'] = area_sqft_df['field'].str.lower()

## Drop the columns that are not needed
high_school_df = high_school_df.drop(columns=['FOP_ratio','Foul-ratio','Foul_pct_of_whole'])


## Match the school names
high_school_df['mat2'] = high_school_df['match'].apply(lambda x: difflib.get_close_matches(x, enroll_df['match'], n=1, cutoff=0.8))
area_sqft_df['mat2'] = area_sqft_df['match'].apply(lambda x: difflib.get_close_matches(x, enroll_df['match'], n=1, cutoff=0.8))
enroll_df['mat2'] = high_school_df['match'].apply(lambda x: difflib.get_close_matches(x, enroll_df['match'], n=1, cutoff=0.8))

enroll_df['mat2'] = enroll_df['mat2'].to_string()
high_school_df['mat2'] = high_school_df['mat2'].to_string()
area_sqft_df['mat2'] = area_sqft_df['mat2'].to_string()
temp_df = high_school_df.merge(area_sqft_df, on='mat2', how = 'outer')
# new_df = temp_df.merge(enroll_df, on='mat2',how='right')

temp_df.head()

#### Merge seems to work
## Output a csv to see what is going on
# temp_df.to_csv('new_df.csv')


## cAN'T make the merge work yet
# outputing the dataframes to csv to see what is going on
# high_school_df.to_csv('high_school_df.csv')
# enroll_df.to_csv('enroll_df.csv')
area_sqft_df.to_csv('area_sqft_df.csv')

# ## Merge the dataframes
# new_df = high_school_df.merge(area_sqft_df, on='mat2',how='right')

# new_df.head()
# high_school_df.head()
# enroll_df.head()
# area_sqft_df.head()

In [ ]:
import difflib 

# high_school_df['school_match'] = high_school_df['School'].apply(lambda x: difflib.get_close_matches(x, enroll_df['school_name']))
# # high_school_df['school_match'] = high_school_df['school_match']
# high_school_df.dtypes

# new_df = high_school_df.join(enroll_df, on='school_name', how='left', lsuffix='_left', rsuffix='_right')
# new_df
enroll_df['school_match'] = enroll_df['school_name']

area_sqft_df['school_match'] = area_sqft_df['field'].apply(lambda x: difflib.get_close_matches(x, enroll_df['school_match']))
new_df = area_sqft_df.join(enroll_df, on='school_match', how='left', lsuffix='_left', rsuffix='_right')

# area_sqft_df['school_match'] = area_sqft_df['school_match']
# area_sqft_df.dtypes

high_school_df.dtypes
# area_sqft_df.dtypes


# new_df = area_sqft_df.join(high_school_df, on='school_match', how='left', lsuffix='_left', rsuffix='_right')
# new_df


# # Merge the dataframes
# out_df = pd.merge(high_school_df, area_sqft_df, on='school_match', how='left')
# out_df



# area_sqft_df['school_match'] = area_sqft_df['field'].apply(lambda x: difflib.get_close_matches(x, enroll_df['school_match']))
# area_sqft_df
# high_school_df.to_csv('data\high_school_df.csv')

# enroll_df['school_match'] = enroll_df['school_name'].apply(lambda x: difflib.get_close_matches(x, high_school_df['School']))
# enroll_df
# #create duplicate column to retain team name from df2
# high_school_df['team'] = high_school_df['School']

# #convert team name in df2 to team name it most closely matches in df1
# high_school_df['school_name'] = high_school_df['team'].apply(lambda x: difflib.get_close_matches(x, enroll_df['school_name']))

# #merge the DataFrames into one
# out_df = pd.merge(high_school_df, enroll_df, on='school_name', how='left')

# #view final DataFrame
# print(out_df)